In [ ]:
from google.colab import drive
import cv2 
import matplotlib.pyplot as plt

In [ ]:
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
info = []

with open("/content/drive/My Drive/255 Project/all-mias/Info.txt") as f:
    lines = f.readlines()
    for line in lines:
      #print(line)
      if line.startswith("mdb", 0, len(line)):
        info += [line.split()]
        print(line.split())


['mdb001', 'G', 'CIRC', 'B', '535', '425', '197']
['mdb002', 'G', 'CIRC', 'B', '522', '280', '69']
['mdb003', 'D', 'NORM']
['mdb004', 'D', 'NORM']
['mdb005', 'F', 'CIRC', 'B', '477', '133', '30']
['mdb005', 'F', 'CIRC', 'B', '500', '168', '26']
['mdb006', 'F', 'NORM']
['mdb007', 'G', 'NORM']
['mdb008', 'G', 'NORM']
['mdb009', 'F', 'NORM']
['mdb010', 'F', 'CIRC', 'B', '525', '425', '33']
['mdb011', 'F', 'NORM']
['mdb012', 'F', 'CIRC', 'B', '471', '458', '40']
['mdb013', 'G', 'MISC', 'B', '667', '365', '31']
['mdb014', 'G', 'NORM']
['mdb015', 'G', 'CIRC', 'B', '595', '864', '68']
['mdb016', 'G', 'NORM']
['mdb017', 'G', 'CIRC', 'B', '547', '573', '48']
['mdb018', 'G', 'NORM']
['mdb019', 'G', 'CIRC', 'B', '653', '477', '49']
['mdb020', 'G', 'NORM']
['mdb021', 'G', 'CIRC', 'B', '493', '125', '49']
['mdb022', 'G', 'NORM']
['mdb023', 'G', 'CIRC', 'M', '538', '681', '29']
['mdb024', 'G', 'NORM']
['mdb025', 'F', 'CIRC', 'B', '674', '443', '79']
['mdb026', 'F', 'NORM']
['mdb027', 'F', 'NORM']
['

In [ ]:
import pandas as pd
#preprocess
df = pd.DataFrame(info, columns = ['Reference_number', 'Background_tissue', 'Class_of_abnormality', 'Severity_of_abnormality', 'x', 'y', 'radius'])
df_clean = df[df.x != '*NOTE']
df_clean = df_clean[df_clean.Reference_number != 'mdb059']
df_clean.fillna(0, inplace=True)
df_clean['x'] = df_clean['x'].astype(int)
df_clean['y'] = df_clean['y'].astype(int)
df_clean['radius'] = df_clean['radius'].astype(int)


df_clean['x_min'] = df_clean.apply(lambda row: row.x - row.radius , axis = 1)
df_clean['y_min'] = df_clean.apply(lambda row: 1024 - row.y - row.radius , axis = 1)
df_clean['x_max'] = df_clean.apply(lambda row: row.x + row.radius , axis = 1)
df_clean['y_max'] = df_clean.apply(lambda row: 1024 - row.y + row.radius , axis = 1)



df_clean.loc[df["Severity_of_abnormality"] == 'B', "Severity_of_abnormality"] = 1
df_clean.loc[df["Severity_of_abnormality"] == 'M', "Severity_of_abnormality"] = 2
df_clean.loc[df["Class_of_abnormality"] == 'CALC', "Class_of_abnormality"] = 0
df_clean.loc[df["Class_of_abnormality"] == 'CIRC', "Class_of_abnormality"] = 1
df_clean.loc[df["Class_of_abnormality"] == 'SPIC', "Class_of_abnormality"] = 2
df_clean.loc[df["Class_of_abnormality"] == 'MISC', "Class_of_abnormality"] = 3
df_clean.loc[df["Class_of_abnormality"] == 'ARCH', "Class_of_abnormality"] = 4
df_clean.loc[df["Class_of_abnormality"] == 'ASYM', "Class_of_abnormality"] = 5
df_clean.loc[df["Class_of_abnormality"] == 'NORM', "Class_of_abnormality"] = 6
df_clean.loc[df["Class_of_abnormality"] == 'NORM', "y_max"] = 0
df_clean.loc[df["Class_of_abnormality"] == 'NORM', "y_min"] = 0
df_clean

,Reference_number,Background_tissue,Class_of_abnormality,Severity_of_abnormality,x,y,radius,x_min,y_min,x_max,y_max
0,mdb001,G,1,1,535,425,197,338,402,732,796
1,mdb002,G,1,1,522,280,69,453,675,591,813
2,mdb003,D,6,0,0,0,0,0,0,0,0
3,mdb004,D,6,0,0,0,0,0,0,0,0
4,mdb005,F,1,1,477,133,30,447,861,507,921
...,...,...,...,...,...,...,...,...,...,...,...
325,mdb318,D,6,0,0,0,0,0,0,0,0
326,mdb319,D,6,0,0,0,0,0,0,0,0
327,mdb320,D,6,0,0,0,0,0,0,0,0
328,mdb321,D,6,0,0,0,0,0,0,0,0


In [ ]:
 def to_yolo(title, label ,xmin, ymin, xmax, ymax, img_w, img_h):
    dw = 1./(img_w)
    dh = 1./(img_h)
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (title,label,x,y,w,h)

In [ ]:
img_w, img_h = 1024, 1024
 
yolo = pd.DataFrame(to_yolo(df_clean.Reference_number ,df_clean.Class_of_abnormality ,df_clean.x_min, df_clean.y_min, df_clean.x_max ,df_clean.y_max, img_w, img_h))

In [ ]:
yolo

,0,1,2,3,4,5,6,7,8,9,...,320,321,322,323,324,325,326,327,328,329
Reference_number,mdb001,mdb002,mdb003,mdb004,mdb005,mdb005,mdb006,mdb007,mdb008,mdb009,...,mdb313,mdb314,mdb315,mdb316,mdb317,mdb318,mdb319,mdb320,mdb321,mdb322
Class_of_abnormality,1,1,6,6,1,1,6,6,6,6,...,6,3,1,6,6,6,6,6,6,6
Unnamed 0,0.522461,0.509766,0.0,0.0,0.46582,0.488281,0.0,0.0,0.0,0.0,...,0.0,0.505859,0.503906,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Unnamed 1,0.584961,0.726562,0.0,0.0,0.870117,0.835938,0.0,0.0,0.0,0.0,...,0.0,0.813477,0.563477,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Unnamed 2,0.384766,0.134766,0.0,0.0,0.058594,0.050781,0.0,0.0,0.0,0.0,...,0.0,0.076172,0.181641,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Unnamed 3,0.384766,0.134766,0.0,0.0,0.058594,0.050781,0.0,0.0,0.0,0.0,...,0.0,0.076172,0.181641,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Transpose
yolo = yolo.T
yolo.columns
yolo.columns = yolo.columns.str.replace(' ', '')

In [ ]:
yolo.rename(columns = {'Unnamed0':'x'}, inplace = True)
yolo.rename(columns = {'Unnamed1':'y'}, inplace = True)
yolo.rename(columns = {'Unnamed2':'w'}, inplace = True)
yolo.rename(columns = {'Unnamed3':'h'}, inplace = True)
yolo

,Reference_number,Class_of_abnormality,x,y,w,h
0,mdb001,1,0.522461,0.584961,0.384766,0.384766
1,mdb002,1,0.509766,0.726562,0.134766,0.134766
2,mdb003,6,0.0,0.0,0.0,0.0
3,mdb004,6,0.0,0.0,0.0,0.0
4,mdb005,1,0.46582,0.870117,0.058594,0.058594
...,...,...,...,...,...,...
325,mdb318,6,0.0,0.0,0.0,0.0
326,mdb319,6,0.0,0.0,0.0,0.0
327,mdb320,6,0.0,0.0,0.0,0.0
328,mdb321,6,0.0,0.0,0.0,0.0


In [ ]:
yolo_needed = yolo[yolo.Class_of_abnormality != 6]
yolo_needed

,Reference_number,Class_of_abnormality,x,y,w,h
0,mdb001,1,0.522461,0.584961,0.384766,0.384766
1,mdb002,1,0.509766,0.726562,0.134766,0.134766
4,mdb005,1,0.46582,0.870117,0.058594,0.058594
5,mdb005,1,0.488281,0.835938,0.050781,0.050781
10,mdb010,1,0.512695,0.584961,0.064453,0.064453
...,...,...,...,...,...,...
281,mdb274,3,0.124023,0.506836,0.240234,0.240234
297,mdb290,1,0.329102,0.655273,0.087891,0.087891
319,mdb312,3,0.234375,0.743164,0.039062,0.039062
321,mdb314,3,0.505859,0.813477,0.076172,0.076172


#Write TXT

In [ ]:
#import os
#os.mkdir("/content/drive/MyDrive/255 Project/yolo_anno")

In [ ]:
%cd "/content/drive/MyDrive/255 Project/yolo_anno"

/content/drive/MyDrive/255 Project/yolo_anno


In [ ]:
for Reference_number, group in yolo_needed.groupby(['Reference_number']):
  group.to_csv(f'{Reference_number}.txt',sep=" ",header=False, index=False, columns=['Class_of_abnormality','x','y','w','h'])

# Move NORM file to another directory


In [ ]:
source = '/content/drive/MyDrive/255 Project/mias_dataset'
destination = '/content/drive/MyDrive/255 Project/Augmentation/NON_NORM Image'

In [ ]:
yolo_NORM = yolo[yolo.Class_of_abnormality == 6]
yolo_NORM

,Reference_number,Class_of_abnormality,x,y,w,h
2,mdb003,6,0.0,0.0,0.0,0.0
3,mdb004,6,0.0,0.0,0.0,0.0
6,mdb006,6,0.0,0.0,0.0,0.0
7,mdb007,6,0.0,0.0,0.0,0.0
8,mdb008,6,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
325,mdb318,6,0.0,0.0,0.0,0.0
326,mdb319,6,0.0,0.0,0.0,0.0
327,mdb320,6,0.0,0.0,0.0,0.0
328,mdb321,6,0.0,0.0,0.0,0.0


In [ ]:
yolo_NON_NORM = yolo[yolo.Class_of_abnormality != 6]
yolo_NON_NORM

,Reference_number,Class_of_abnormality,x,y,w,h
0,mdb001,1,0.522461,0.584961,0.384766,0.384766
1,mdb002,1,0.509766,0.726562,0.134766,0.134766
4,mdb005,1,0.46582,0.870117,0.058594,0.058594
5,mdb005,1,0.488281,0.835938,0.050781,0.050781
10,mdb010,1,0.512695,0.584961,0.064453,0.064453
...,...,...,...,...,...,...
281,mdb274,3,0.124023,0.506836,0.240234,0.240234
297,mdb290,1,0.329102,0.655273,0.087891,0.087891
319,mdb312,3,0.234375,0.743164,0.039062,0.039062
321,mdb314,3,0.505859,0.813477,0.076172,0.076172


In [ ]:
import glob
import os
import shutil

In [ ]:
#yolo_NORM.Reference_number
yolo_NON_NORM.Reference_number

0      mdb001
1      mdb002
4      mdb005
5      mdb005
10     mdb010
        ...  
281    mdb274
297    mdb290
319    mdb312
321    mdb314
322    mdb315
Name: Reference_number, Length: 119, dtype: object

In [ ]:
#NORM_Ref = list(yolo_NORM.Reference_number)
#print(NORM_Ref)

NON_NORM_Ref = list(yolo_NON_NORM.Reference_number)
print(NON_NORM_Ref)

['mdb001', 'mdb002', 'mdb005', 'mdb005', 'mdb010', 'mdb012', 'mdb013', 'mdb015', 'mdb017', 'mdb019', 'mdb021', 'mdb023', 'mdb025', 'mdb028', 'mdb030', 'mdb032', 'mdb058', 'mdb063', 'mdb069', 'mdb072', 'mdb075', 'mdb080', 'mdb081', 'mdb083', 'mdb090', 'mdb091', 'mdb092', 'mdb095', 'mdb097', 'mdb099', 'mdb102', 'mdb104', 'mdb105', 'mdb107', 'mdb110', 'mdb111', 'mdb115', 'mdb117', 'mdb120', 'mdb121', 'mdb124', 'mdb125', 'mdb126', 'mdb127', 'mdb130', 'mdb132', 'mdb132', 'mdb134', 'mdb141', 'mdb142', 'mdb144', 'mdb144', 'mdb145', 'mdb148', 'mdb150', 'mdb152', 'mdb155', 'mdb158', 'mdb160', 'mdb163', 'mdb165', 'mdb167', 'mdb170', 'mdb171', 'mdb175', 'mdb178', 'mdb179', 'mdb181', 'mdb184', 'mdb186', 'mdb188', 'mdb190', 'mdb191', 'mdb193', 'mdb195', 'mdb198', 'mdb199', 'mdb202', 'mdb204', 'mdb206', 'mdb207', 'mdb209', 'mdb211', 'mdb212', 'mdb213', 'mdb214', 'mdb218', 'mdb219', 'mdb222', 'mdb223', 'mdb223', 'mdb226', 'mdb226', 'mdb226', 'mdb227', 'mdb231', 'mdb236', 'mdb238', 'mdb239', 'mdb239',

In [ ]:
for i in NON_NORM_Ref:
  allfiles = glob.glob(os.path.join(source, i+".jpg"), recursive=True)
  print("Files to move", allfiles)
  for file_path in allfiles:
      dst_path = os.path.join(destination, os.path.basename(file_path))
      shutil.move(file_path, dst_path)
      print(f"Moved {file_path} -> {dst_path}")


Files to move ['/content/drive/MyDrive/255 Project/mias_dataset/mdb001.jpg']
Moved /content/drive/MyDrive/255 Project/mias_dataset/mdb001.jpg -> /content/drive/MyDrive/255 Project/Augmentation/NON_NORM Image/mdb001.jpg
Files to move ['/content/drive/MyDrive/255 Project/mias_dataset/mdb002.jpg']
Moved /content/drive/MyDrive/255 Project/mias_dataset/mdb002.jpg -> /content/drive/MyDrive/255 Project/Augmentation/NON_NORM Image/mdb002.jpg
Files to move ['/content/drive/MyDrive/255 Project/mias_dataset/mdb005.jpg']
Moved /content/drive/MyDrive/255 Project/mias_dataset/mdb005.jpg -> /content/drive/MyDrive/255 Project/Augmentation/NON_NORM Image/mdb005.jpg
Files to move []
Files to move ['/content/drive/MyDrive/255 Project/mias_dataset/mdb010.jpg']
Moved /content/drive/MyDrive/255 Project/mias_dataset/mdb010.jpg -> /content/drive/MyDrive/255 Project/Augmentation/NON_NORM Image/mdb010.jpg
Files to move ['/content/drive/MyDrive/255 Project/mias_dataset/mdb012.jpg']
Moved /content/drive/MyDrive/2